In [28]:
!git clone https://github.com/antonisa/lang2vec
%cd lang2vec
!python3 setup.py install

Cloning into 'lang2vec'...
remote: Enumerating objects: 140, done.
remote: Total 140 (delta 0), reused 0 (delta 0), pack-reused 140 (from 1)
Receiving objects: 100% (140/140), 71.38 MiB | 7.04 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/Users/kaustubhkislay/LLM-Language-Learning/notebooks/lang2vec


/Users/kaustubhkislay/miniconda3/envs/llmlearn/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


running install
/Users/kaustubhkislay/miniconda3/envs/llmlearn/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:90: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        By 2025-Oct-31, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
running build
running build_py
creating build/lib/lang2vec
copying lang2vec/__init__.py -> build/lib/lang2vec
copying lang2vec/lang2vec.py -> build/lib/lang2vec
creating build/lib/lang2vec/data
copying lang2vec/data/feature_averag

In [3]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine as cosdist
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [4]:
import lang2vec.lang2vec as l2v


In [13]:
l2v.syntactic_distance('eng', 'spa')

0.4

In [ ]:
l2v.syntactic_distance('eng','')
l2v.geographic_distance('eng','')
l2v.phonological_distance('eng','')
l2v.genetic_distance('eng','')
l2v.inventory_distance('eng','')
l2v.featural_distance('eng','')

1.0

In [26]:
l2v.distance(['syntactic','geographic'], 'frr', 'dan', 'hin')

[array([[0.  , 0.66, 0.58],
        [0.66, 0.  , 0.67],
        [0.58, 0.67, 0.  ]]),
 array([[0. , 0. , 0.3],
        [0. , 0. , 0.3],
        [0.3, 0.3, 0. ]])]

In [30]:
# Example for the genetic distance space
df = pd.read_csv("/Users/kaustubhkislay/LLM-Language-Learning/notebooks/lang2vec/lang2vec/data/distances/GENETIC.csv", index_col=0)
langs = df.index.tolist()
print(len(langs), "languages available")
print(langs[:10])  # first 10 codes


8070 languages available
['aap', 'aca', 'acu', 'adc', 'adw', 'agr', 'ake', 'ako', 'alc', 'amc']


In [31]:
for space in ["SYNTACTIC", "PHONOLOGICAL", "INVENTORY", "FEATURAL"]:
    path = f"/Users/kaustubhkislay/LLM-Language-Learning/notebooks/lang2vec/lang2vec/data/distances/{space}.csv"
    df = pd.read_csv(path, index_col=0)
    print(f"{space}: {len(df)} languages")


SYNTACTIC: 8070 languages
PHONOLOGICAL: 8070 languages
INVENTORY: 8070 languages
FEATURAL: 8070 languages


In [32]:
import pandas as pd
import glob

# Define spaces to include in the aggregate
spaces = ["SYNTACTIC", "PHONOLOGICAL", "INVENTORY", "FEATURAL"]

# Path to distance data (adjust if your clone is in a different location)
base_path = "/Users/kaustubhkislay/LLM-Language-Learning/notebooks/lang2vec/lang2vec/data/distances"

# Load all distance matrices into a dictionary
distances = {}
for space in spaces:
    path = f"{base_path}/{space}.csv"
    df = pd.read_csv(path, index_col=0)
    distances[space] = df

# Target language: English ('eng')
target_lang = 'eng'

# Compute average distance for each language
avg_distances = {}

for lang in distances['SYNTACTIC'].columns:
    scores = []
    for space, df in distances.items():
        # Only include if both languages exist in that matrix
        if target_lang in df.index and lang in df.columns:
            scores.append(df.loc[target_lang, lang])
    if len(scores) == len(spaces):  # ensure all spaces are valid
        avg_distances[lang] = sum(scores) / len(scores)

# Convert to DataFrame and sort by descending average distance
results = pd.DataFrame.from_dict(avg_distances, orient='index', columns=['AverageDistance'])
results = results.sort_values(by='AverageDistance', ascending=False)

# Show top 10 languages most distant from English
print(results.head(10))


     AverageDistance
pap         0.899575
mww         0.899575
zhb         0.899575
otm         0.881825
pri         0.879875
ots         0.879875
cnl         0.879875
ott         0.879875
hvn         0.872550
nki         0.869925


In [43]:
high_resource_langs = [
    'eng', 'spa', 'fra', 'deu', 'ita', 'por', 'rus', 'zho', 'jpn',
    'kor', 'ara', 'hin', 'tur', 'nld', 'pol', 'vie', 'tha', 'ind',
    'msa', 'ukr', 'ell', 'ron', 'bul', 'swe', 'fin', 'dan', 'ces'
]

In [49]:
import pandas as pd

# Assuming `results` is your sorted DataFrame:
# Language code as index, "AverageDistance" as column.

def find_nearest(df, target_value):
    """Find the row with AverageDistance closest to target_value."""
    idx = (df['AverageDistance'] - target_value).abs().idxmin()
    return idx, df.loc[idx, 'AverageDistance']

# Pick the three approximate targets
# Filter to high-resource only
filtered = results.loc[results.index.isin(high_resource_langs)]

# Then pick approximate targets
for t in [0.6, 0.5, 0.4]:
    lang, dist = find_nearest(filtered, t)
    print(f"≈{t}: {lang} ({dist:.3f})")

≈0.6: ces (0.597)
≈0.5: swe (0.510)
≈0.4: deu (0.401)
